In [ ]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
import torch
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset

# 1. Load the Model
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = None, # Will default to torch.bfloat16 if available
)

# 2. Configure LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank of the adapters. A common choice.
    lora_alpha = 16, # A scaling factor for the adapters.
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 42,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

print("Unsloth model configured for 4-bit LoRA fine-tuning!")


In [ ]:
# Load Datasets and Merge them
from datasets import load_dataset, concatenate_datasets

def load_and_merge_finance_datasets():
    print("Loading gbharti/wealth-alpaca_lora dataset...")
    wealth_ds = load_dataset("gbharti/wealth-alpaca_lora", split="train")

    print("Loading Josephgflowers/Finance-Instruct-500k dataset...")
    finance_ds = load_dataset("Josephgflowers/Finance-Instruct-500k", split="train")

    def preprocess_wealth_alpaca(example):
        if example.get('input'):
            example['instruction'] = f"{example['instruction']}\n{example['input']}"
        return {"instruction": example["instruction"], "output": example["output"]}

    def preprocess_finance_instruct(example):
        # The output should come from the 'assistant' column in the dataset
        return {"instruction": example["user"], "output": example["assistant"]}

    wealth_ds = wealth_ds.map(preprocess_wealth_alpaca, remove_columns=wealth_ds.column_names)
    finance_ds = finance_ds.map(preprocess_finance_instruct, remove_columns=finance_ds.column_names)

    print("Merging the datasets...")
    merged_dataset = concatenate_datasets([wealth_ds, finance_ds])
    return merged_dataset

merged_dataset = load_and_merge_finance_datasets()

In [ ]:
# Prepare Data for Qwen3 ChatML format

# We create a new column 'text' that contains the formatted prompt.
# SFTTrainer will then use this column for training.
def formatting_prompts_func(example):
    messages = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["output"]},
    ]
    # The tokenizer formats the messages into the required ChatML string.
    # We don't tokenize here, just create the formatted text string.
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return { "text": text }

dataset = merged_dataset.map(formatting_prompts_func)

print("\n--- Formatted Dataset Example ---")
print(dataset[0]["text"])

In [ ]:
# Configure LoRA and Start Training
from trl import SFTTrainer
from transformers import TrainingArguments

# --- Training Arguments ---
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4, # Effective batch size = 2 * 4 = 8
    warmup_steps = 10,
    max_steps = 300,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 42,
    output_dir = "outputs",
)

# --- Initialize Trainer ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # Point trainer to our formatted 'text' column
    max_seq_length = max_seq_length,
    args = training_args,
)

# --- Start Fine-tuning ---
print("Starting the fine-tuning process...")
trainer.train()
print("Fine-tuning complete!")

In [ ]:
# Inference and Saving the Model

print("\n--- Running Inference ---")
from transformers import pipeline

# Use Unsloth's fast inference pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Create a test prompt
messages = [
    {"role": "user", "content": "What are the main risks associated with investing in emerging markets?"},
]

# Get the response
outputs = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.7, top_p=0.95)
print(outputs[0]['generated_text'])


In [ ]:
# Save the Adapters and Push to Hugging Face Hub
from huggingface_hub import notebook_login

# Save the fine-tuned LoRA adapters
print("\n--- Saving LoRA Adapters ---")
model.save_pretrained("qwen3_30b_finance_lora")
tokenizer.save_pretrained("qwen3_30b_finance_lora")
print("Model adapters saved to 'qwen3_30b_finance_lora'")

# Log in to Hugging Face Hub
notebook_login()

# Push the model adapters and tokenizer to the Hub
repo_name = "huseyincavus/qwen3-30b-finance-lora"

print(f"\n--- Pushing LoRA Adapters to Hugging Face Hub ({repo_name}) ---")
model.push_to_hub(repo_name, token = True)
tokenizer.push_to_hub(repo_name, token = True)
print("Model adapters and tokenizer pushed to Hugging Face Hub!")